# Choroid Plexus Segmentation Training via Auto3DSeg

In [20]:
import os
import json
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import glob
import random
import platform
from monai.apps.auto3dseg import AutoRunner
from monai.config import print_config
import importlib
from dataclasses import asdict
from loguru import logger
import sys

from reload_recursive import reload_recursive

import mri_data
import monai_training

print_config()

MONAI version: 1.4.0
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu124
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 46a5272196a6c2590ca2589029eed8e4d56ff008
MONAI __file__: /home/<username>/.virtualenvs/monai/lib/python3.12/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: 1.14.1
Pillow version: 11.0.0
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.66.6
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 6.1.0
pandas version: 2.2.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: 2.17.2
pynrrd version: NOT INSTALLED or UNKNOWN VERSI

In [21]:
reload_recursive(monai_training)
reload_recursive(mri_data)

from mri_data.file_manager import scan_3Tpioneer_bids, filter_first_ses
from monai_training.preprocess import DataSetProcesser
from monai_training import training, preprocess

In [22]:
logger.remove()
logger.add(sys.stderr, level="DEBUG")

2

In [ ]:
load_data = True

In [27]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")
msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

dataroot = drive_root / "3Tpioneer_bids"
work_dir_name = "choroid_pineal_pituitary1"
work_dir = training_work_dirs / work_dir_name
modalities = ["flair", "t1"]
labels = ["choroid_t1_flair", "pineal", "pituitary"]

## Prep the database

Get the data and labels organized

In [28]:
if load_data:
    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, 'r') as f:
        datalist = json.load(f)

    dataset = preprocess.parse_datalist(datalist_file, dataroot)

In [29]:
subs = sorted([int(scan.subid) for scan in dataset])
print(subs)

[1010, 1011, 1019, 1033, 1065, 1080, 1109, 1119, 1163, 1188, 1191, 1234, 1259, 1265, 1272, 1280, 1293, 1321, 1355, 1437, 1486, 1498, 1518, 1547, 1548, 2081, 2083, 2144]


In [30]:
if not load_data:
    dataset_proc = DataSetProcesser.new_dataset(dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses])
    dataset_proc.prepare_images(modalities)
    dataset_proc.prepare_labels(labels, suffix_list=["CH", "SRS", "ED", "DT"])

    dataset = dataset_proc.dataset
    dataset.sort()

2024-11-05 18:22:02.576 | INFO     | monai_training.preprocess:prepare_images:107 - Prepare Images
 37%|███▋      | 212/575 [00:00<00:01, 205.25it/s]2024-11-05 18:22:03.668 | INFO     | mri_data.utils:merge_images:29 - fslmerge -a /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.t1.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz


Image Exception : #22 :: Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated


2024-11-05 18:22:08.807 | ERROR    | monai_training.preprocess:prepare_images:147 - Something went wrong merging images for <bound method Scan.info of Scan(subid='1196', sesid='20161004', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids'), _root=PosixPath('/media/smbshare/3Tpioneer_bids/sub-ms1196/ses-20161004'), image=None, label=None, cond=None, id=24112560784)>
 44%|████▍     | 253/575 [00:06<00:18, 17.54it/s] 

/home/srs-9/fsl/share/fsl/bin/fslmerge: line 2: 35395 Aborted                 (core dumped) /home/srs-9/fsl/bin/fslmerge "$@"


 83%|████████▎ | 480/575 [00:07<00:00, 166.30it/s]2024-11-05 18:22:10.124 | INFO     | mri_data.utils:merge_images:29 - fslmerge -a /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.t1.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/t1.nii.gz


Image Exception : #22 :: Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated


2024-11-05 18:22:11.707 | ERROR    | monai_training.preprocess:prepare_images:147 - Something went wrong merging images for <bound method Scan.info of Scan(subid='2120', sesid='20170920', _dataroot=PosixPath('/media/smbshare/3Tpioneer_bids'), _root=PosixPath('/media/smbshare/3Tpioneer_bids/sub-ms2120/ses-20170920'), image=None, label=None, cond=None, id=42762350400)>
 91%|█████████ | 521/575 [00:09<00:01, 45.91it/s] 2024-11-05 18:22:11.867 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1245, sesid=20220219)


/home/srs-9/fsl/share/fsl/bin/fslmerge: line 2: 35426 Aborted                 (core dumped) /home/srs-9/fsl/bin/fslmerge "$@"


100%|██████████| 575/575 [00:09<00:00, 60.26it/s]
2024-11-05 18:22:12.119 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
2024-11-05 18:22:12.119 | DEBUG    | monai_training.preprocess:prepare_labels:178 - choroid_t1_flair.pineal.pituitary.nii.gz
  0%|          | 0/565 [00:00<?, ?it/s]2024-11-05 18:22:12.127 | DEBUG    | monai_training.preprocess:prepare_labels:196 - Need to create choroid_t1_flair.pineal.pituitary.nii.gz
2024-11-05 18:22:12.127 | INFO     | mri_data.utils:combine_labels:86 - Combining labels for Scan(subid=1027, sesid=20170127)
2024-11-05 18:22:12.134 | DEBUG    | mri_data.file_manager:find_label:401 - Testing choroid_t1_flair-ch.nii.gz
2024-11-05 18:22:12.134 | DEBUG    | mri_data.file_manager:find_label:401 - Testing choroid_t1_flair-srs.nii.gz
2024-11-05 18:22:12.134 | DEBUG    | mri_data.file_manager:find_label:401 - Testing choroid_t1_flair-ed.nii.gz
2024-11-05 18:22:12.134 | DEBUG    | mri_data.file_manager:find_label:401 - Testing cho

In [31]:
if not load_data:
    dataset = training.assign_conditions(dataset, 0.2)
    dataset[0]

In [32]:
if not load_data:
    training_data = []
    test_data = []

    for scan in dataset:
        if scan.cond == 'tr':
            training_data.append({"image": scan.image, "label": scan.label})
        elif scan.cond == 'ts':
            test_data.append(scan.image)

## Review

In [ ]:
def display_slices(scan):
    img1 = nib.load(scan.image_path)
    img2 = nib.load(scan.label_path)

    data1 = img1.get_fdata()[:,:,:]
    data2 = img2.get_fdata()

    slice_sums = np.sum(data2, axis=(0, 1))
    
    print(slice_sums)

    max_slice_index = np.argmax(slice_sums)
    print(f"Max slice: {max_slice_index}")

    slice1 = data1[:, :, max_slice_index]
    slice2 = data2[:, :, max_slice_index]    

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(slice1, cmap='gray')
    plt.title(f"Image 1 - Slice {max_slice_index}")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(slice2, cmap='gray')
    plt.title(f"Image 2 - Slice {max_slice_index}")
    plt.axis('off')
    plt.show()


In [ ]:
# display_slices(dataset[20])

In the original code, they include labels in the test data as well. Also they have a function that checks that there is nonzero number of voxels in the label

In [33]:
#? I don't know why they put labels for the test data. the brats segmentation code didn't.
# train_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in train_exams]
# test_data = [{'image': path + '/flair.nii.gz', 'label': path + '/flair_chp_mask_qced.nii.gz'} for path in test_exams]
if not load_data:
    train_data = []
    test_data = []
    for scan in dataset:
        if scan.cond == 'tr' and scan.has_label:
            train_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})
        elif scan.cond == 'ts' and scan.has_label():
            test_data.append({"image": str(scan.image_path), "label": str(scan.label_path)})


    print(f"Train num total: {len(train_data)}")
    print(f"Test num: {len(test_data)}")

Train num total: 24
Test num: 5


Create and save datalist

In [34]:
if not load_data:
    n_folds = 5
    datalist = {
        "testing": test_data,
        "training": [{"fold": i % n_folds, "image": c["image"], "label": c["label"]} for i,c in enumerate(train_data)]
    }

    if not os.path.isdir(work_dir):
        os.makedirs(work_dir)

    # dataroot_dir = "/mnt/h"
    # if not os.path.isdir(dataroot_dir):
    #     os.makedirs(dataroot_dir)

    datalist_file = os.path.join(work_dir, "datalist.json")
    with open(datalist_file, "w") as f:
        json.dump(datalist, f)

Load datalist

In [35]:
datalist['testing']

[{'image': '/media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/choroid_t1_flair-CH.pineal-CH.pituitary-CH.nii.gz'},
 {'image': '/media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/choroid_t1_flair-ED.pineal-CH.pituitary-CH.nii.gz'},
 {'image': '/media/smbshare/3Tpioneer_bids/sub-ms1293/ses-20161129/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms1293/ses-20161129/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz'},
 {'image': '/media/smbshare/3Tpioneer_bids/sub-ms1355/ses-20210104/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms1355/ses-20210104/choroid_t1_flair-ED.pineal-SRS.pituitary-CH.nii.gz'},
 {'image': '/media/smbshare/3Tpioneer_bids/sub-ms2144/ses-20190422/flair.t1.nii.gz',
  'label': '/media/smbshare/3Tpioneer_bids/sub-ms2144/ses-20190422/choroid_t1_flair-ED.pineal

In [36]:
missing_images = []
missing_labels = []
for scan in dataset:
    if not scan.image_path.is_file():
        missing_images.append(scan.image_path)
    if not scan.label_path.is_file():
        missing_images.append(scan.label_path)
print(missing_images)
print(missing_labels)

[]
[]


In [37]:
runner = AutoRunner(
    work_dir=work_dir,
    algos=["swinunetr"],
    input={
        "modality": "MRI",
        "datalist": str(datalist_file),
        "dataroot": str(dataroot),
    },
)

2024-11-05 18:22:55,008 - INFO - AutoRunner using work directory /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary1
2024-11-05 18:22:55,009 - INFO - Found num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary1/datalist.json.
2024-11-05 18:22:55,010 - INFO - Setting num_fold 5 based on the input datalist /home/srs-9/Projects/ms_mri/training_work_dirs/choroid_pineal_pituitary1/datalist.json.
2024-11-05 18:22:55,017 - INFO - Using user defined command running prefix , will override other settings


/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/torch/cuda/__init__.py:716: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [38]:
max_epochs = 100

train_param = {
    "num_epochs_per_validation": 1,
    #"num_images_per_batch": 2,
    "num_epochs": max_epochs,
    "num_warmup_epochs": 1,
}
runner.set_training_params(train_param)

In [39]:
runner.run()

2024-11-05 18:22:58,228 - INFO - Running data analysis...
2024-11-05 18:22:58,233 - INFO - Found 0 GPUs for data analyzing!
2024-11-05 18:22:58,237 - INFO - device=cuda:0 but CUDA device is not available, using CPU instead.


  0%|          | 0/24 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
scan_path = "/mnt/t/Data/3Tpioneer_bids/sub-ms1001/ses-20170215/proc/lesion_index.t3m20-mni_reg.nii.gz"
img = nib.load(scan_path)
